In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


Phase 2

In [ ]:
import os
os.listdir()


In [ ]:
import pandas as pd

train = pd.read_csv("train.csv")
features = pd.read_csv("features.csv")
stores = pd.read_csv("stores.csv")

train.head()


In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

session.write_pandas(train, "TRAIN_RAW", auto_create_table=True, overwrite=True)
session.write_pandas(features, "FEATURES_RAW", auto_create_table=True, overwrite=True)
session.write_pandas(stores, "STORES_RAW", auto_create_table=True, overwrite=True)


In [ ]:
SELECT 'TRAIN_RAW' AS T, COUNT(*) AS N FROM TRAIN_RAW
UNION ALL
SELECT 'FEATURES_RAW', COUNT(*) FROM FEATURES_RAW
UNION ALL
SELECT 'STORES_RAW', COUNT(*) FROM STORES_RAW;


In [ ]:
DESC TABLE TRAIN_RAW;


In [ ]:
CREATE OR REPLACE TABLE STORE_WEEKLY_SALES AS
SELECT
    t."Date" AS DS,
    SUM(t."Weekly_Sales") AS Y,
    MAX(IFF(t."IsHoliday", 1, 0)) AS IS_HOLIDAY,
    AVG(f."Temperature") AS TEMPERATURE,
    AVG(f."Fuel_Price") AS FUEL_PRICE,
    AVG(f."CPI") AS CPI,
    AVG(f."Unemployment") AS UNEMPLOYMENT
FROM TRAIN_RAW t
LEFT JOIN FEATURES_RAW f
  ON t."Store" = f."Store"
 AND t."Date"  = f."Date"
WHERE t."Store" = 1   
GROUP BY t."Date"
ORDER BY DS;


In [ ]:
SELECT COUNT(*) FROM STORE_WEEKLY_SALES;


In [ ]:
SELECT * FROM STORE_WEEKLY_SALES
ORDER BY DS
LIMIT 10;


In [ ]:
from snowflake.snowpark.context import get_active_session
import pandas as pd
import matplotlib.pyplot as plt

session = get_active_session()

df = session.table("STORE_WEEKLY_SALES").to_pandas()
df["DS"] = pd.to_datetime(df["DS"])
df = df.sort_values("DS").set_index("DS")

df.head()


Phase 3

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure()
plt.plot(df.index, df["Y"])
plt.title("Weekly Total Sales - Store 1")
plt.xlabel("Week")
plt.ylabel("Total Weekly Sales")

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))      # every 3 months
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))      # label format

plt.gcf().autofmt_xdate()   # rotates/aligns labels
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

df["rolling_mean_12"] = df["Y"].rolling(window=12).mean()

plt.figure()
plt.plot(df.index, df["Y"], label="Weekly Sales", alpha=0.5)
plt.plot(df.index, df["rolling_mean_12"], label="12-Week Rolling Mean", linewidth=2)
plt.title("Weekly Sales with 12-Week Rolling Mean")
plt.xlabel("Week")
plt.ylabel("Total Weekly Sales")
plt.legend()

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

plt.gcf().autofmt_xdate()
plt.tight_layout()
plt.show()


In [ ]:
holiday_avg = df.groupby("IS_HOLIDAY")["Y"].mean()
holiday_avg


In [ ]:
holiday_avg.plot(kind="bar")
plt.title("Average Weekly Sales: Holiday vs Non-Holiday")
plt.ylabel("Average Weekly Sales")
plt.xticks([0,1], ["Non-Holiday", "Holiday"], rotation=0)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import STL

# Safety: make sure the index is datetime + sorted (doesn't change your values)
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# STL decomposition (weekly data -> yearly seasonality ~ 52 weeks)
stl = STL(df["Y"], period=52)
result = stl.fit()

# Plot + fix the crowded x-axis labels on all subplots
fig = result.plot()

for ax in fig.axes:
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))     # every 3 months
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))     # YYYY-MM

fig.autofmt_xdate()
plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

adf_result = adfuller(df["Y"])

print("ADF Statistic:", adf_result[0])
print("p-value:", adf_result[1])


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.figure(figsize=(10,4))
plot_acf(df["Y"], lags=60)
plt.show()

plt.figure(figsize=(10,4))
plot_pacf(df["Y"], lags=60)
plt.show()


Phase 4

In [ ]:
from snowflake.snowpark.context import get_active_session
import pandas as pd

session = get_active_session()

# Pull your engineered table into pandas
df = session.table("STORE_WEEKLY_SALES").to_pandas()

# Ensure DS is datetime and used as index
df["DS"] = pd.to_datetime(df["DS"])
df = df.sort_values("DS").set_index("DS")

# Quick sanity check
df.head()

In [ ]:
import numpy as np
import pandas as pd

# 12-week forecast horizon
h = 12

# Target series: weekly on Fridays (matches your data)
y = df["Y"].asfreq("W-FRI")

# Handle any missing weeks (rare but possible)
# (this does NOT change existing values; it only fills missing timestamps if any exist)
y = y.interpolate("time").ffill().bfill()

# Exogenous features for SARIMAX (advanced model)
exog_cols = ["IS_HOLIDAY", "TEMPERATURE", "FUEL_PRICE", "CPI", "UNEMPLOYMENT"]
X = df[exog_cols].asfreq("W-FRI")
X = X.interpolate("time").ffill().bfill()

# Train/test split (last 12 weeks as test)
train_y = y.iloc[:-h]
test_y  = y.iloc[-h:]

train_X = X.loc[train_y.index]
test_X  = X.loc[test_y.index]

# Show shapes + date ranges
train_y.shape, test_y.shape, (train_y.index.min(), train_y.index.max(), test_y.index.min(), test_y.index.max())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.filterwarnings("ignore", category=ValueWarning)  # hides the "inferred frequency W-FRI" spam

from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX

# -----------------------------
# Make sure freq is explicit (prevents warnings + keeps indexing consistent)
# Assumes you already created train_y, test_y, train_X, test_X in the prior cell.
# -----------------------------
train_y = train_y.asfreq("W-FRI")
test_y  = test_y.asfreq("W-FRI")
train_X = train_X.asfreq("W-FRI")
test_X  = test_X.asfreq("W-FRI")

train_y.index.freq = "W-FRI"
test_y.index.freq  = "W-FRI"
train_X.index.freq = "W-FRI"
test_X.index.freq  = "W-FRI"

# -----------------------------
# Metrics helpers
# -----------------------------
def mape(y_true, y_pred):
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    eps = 1e-9
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), eps))) * 100

def eval_metrics(y_true, y_pred, label):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5
    mape_val = mape(y_true, y_pred)
    return {"model": label, "MAE": mae, "RMSE": rmse, "MAPE": mape_val}

# ---------------------------------------
# Walk-forward validation (12-step)
# Expanding window, 1-step ahead repeated
# ---------------------------------------
def walk_forward_ets(train_y, test_y):
    history = train_y.copy()
    preds = []
    for t in range(len(test_y)):
        model = ExponentialSmoothing(history, trend="add", seasonal=None).fit(optimized=True)
        yhat = model.forecast(1).iloc[0]
        preds.append(yhat)
        history = pd.concat([history, pd.Series([test_y.iloc[t]], index=[test_y.index[t]])])
        history.index.freq = "W-FRI"
    return np.array(preds)

def walk_forward_sarima(train_y, test_y, order=(1,0,1), seasonal_order=(0,1,0,52)):
    history = train_y.copy()
    preds = []
    for t in range(len(test_y)):
        model = SARIMAX(
            history,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        ).fit(disp=False)
        yhat = model.forecast(1).iloc[0]
        preds.append(yhat)
        history = pd.concat([history, pd.Series([test_y.iloc[t]], index=[test_y.index[t]])])
        history.index.freq = "W-FRI"
    return np.array(preds)

def walk_forward_sarimax(train_y, test_y, train_X, test_X, order=(1,0,1), seasonal_order=(0,1,0,52)):
    history_y = train_y.copy()
    history_X = train_X.copy()
    preds = []

    for t in range(len(test_y)):
        model = SARIMAX(
            history_y,
            exog=history_X,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        ).fit(disp=False)

        yhat = model.forecast(1, exog=test_X.iloc[[t]]).iloc[0]
        preds.append(yhat)

        history_y = pd.concat([history_y, pd.Series([test_y.iloc[t]], index=[test_y.index[t]])])
        history_X = pd.concat([history_X, test_X.iloc[[t]]])

        history_y.index.freq = "W-FRI"
        history_X.index.freq = "W-FRI"

    return np.array(preds)

# -----------------------------
# 1) Baseline model: ETS
# -----------------------------
ets_preds = walk_forward_ets(train_y, test_y)

# -----------------------------
# 2) Baseline model: SARIMA
# -----------------------------
sarima_order = (1, 0, 1)
sarima_seasonal = (0, 1, 0, 52)
sarima_preds = walk_forward_sarima(train_y, test_y, order=sarima_order, seasonal_order=sarima_seasonal)

# -----------------------------
# 3) Advanced model: SARIMAX (with exogenous features)
# -----------------------------
sarimax_order = (1, 0, 1)
sarimax_seasonal = (0, 1, 0, 52)
sarimax_preds = walk_forward_sarimax(
    train_y, test_y,
    train_X, test_X,
    order=sarimax_order,
    seasonal_order=sarimax_seasonal
)

# -----------------------------
# Evaluation table
# -----------------------------
results = [
    eval_metrics(test_y.values, ets_preds, "ETS (baseline)"),
    eval_metrics(test_y.values, sarima_preds, "SARIMA (baseline)"),
    eval_metrics(test_y.values, sarimax_preds, "SARIMAX + exog (advanced)")
]
eval_df = pd.DataFrame(results).sort_values("RMSE")
print(eval_df)

# -----------------------------
# Plot: actual vs forecasts
# -----------------------------
plt.figure()
plt.plot(train_y.index, train_y.values, label="Train")
plt.plot(test_y.index, test_y.values, label="Test (Actual)")
plt.plot(test_y.index, ets_preds, label="ETS Forecast")
plt.plot(test_y.index, sarima_preds, label="SARIMA Forecast")
plt.plot(test_y.index, sarimax_preds, label="SARIMAX Forecast")
plt.title("Phase 4: Walk-Forward Forecasts (12 weeks)")
plt.xlabel("Week")
plt.ylabel("Total Weekly Sales")
plt.legend()
plt.tight_layout()
plt.show()

# -----------------------------
# Save forecasts to Snowflake table
# -----------------------------
forecast_out = pd.DataFrame({
    "DS": test_y.index,
    "Y_ACTUAL": test_y.values,
    "Y_ETS": ets_preds,
    "Y_SARIMA": sarima_preds,
    "Y_SARIMAX": sarimax_preds
})

session.create_dataframe(forecast_out).write.mode("overwrite").save_as_table("STORE_WEEKLY_FORECASTS_PHASE4")

"Saved to Snowflake table: STORE_WEEKLY_FORECASTS_PHASE4"


In [ ]:
display(eval_df)

best_row = eval_df.iloc[0]
best_model = best_row["model"]
best_rmse = best_row["RMSE"]
best_mae = best_row["MAE"]
best_mape = best_row["MAPE"]

print("Best model (lowest RMSE):", best_model)
print(f"RMSE: {best_rmse:,.2f} | MAE: {best_mae:,.2f} | MAPE: {best_mape:.2f}%")

Phase 5

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timezone

from statsmodels.tsa.statespace.sarimax import SARIMAX

# ----------------------------
# Setup
# ----------------------------
df = df.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()

exog_cols = ["IS_HOLIDAY", "TEMPERATURE", "FUEL_PRICE", "CPI", "UNEMPLOYMENT"]

y = df["Y"].asfreq("W-FRI").interpolate("time").ffill().bfill()
X = df[exog_cols].asfreq("W-FRI").interpolate("time").ffill().bfill()

y.index.freq = "W-FRI"
X.index.freq = "W-FRI"

h = 12  # forecast horizon

# ----------------------------
# Pipeline function
# ----------------------------
def run_forecast_pipeline(y, X, horizon=12, scenario_name="BASELINE", scenario_exog_future=None):
    order = (1, 0, 1)
    seasonal_order = (0, 1, 0, 52)

    model = SARIMAX(
        y,
        exog=X,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    fit = model.fit(disp=False)

    last_date = y.index.max()
    future_index = pd.date_range(start=last_date + pd.Timedelta(weeks=1), periods=horizon, freq="W-FRI")
    future_index.freq = "W-FRI"

    if scenario_exog_future is None:
        last_row = X.iloc[-1]
        exog_future = pd.DataFrame([last_row.values] * horizon, columns=X.columns, index=future_index)
    else:
        exog_future = scenario_exog_future.copy()
        exog_future.index = future_index
        exog_future = exog_future[X.columns]  # enforce same column order

    yhat = fit.forecast(steps=horizon, exog=exog_future)
    yhat = pd.Series(yhat.values, index=future_index, name="Y_PRED")

    run_ts = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S%z")
    out = pd.DataFrame({
        "RUN_TS_UTC": run_ts,
        "SCENARIO": scenario_name,
        "DS": yhat.index,
        "Y_PRED": yhat.values
    })

    for col in exog_future.columns:
        out[col] = exog_future[col].values

    coef_tbl = pd.DataFrame({
        "PARAMETER": fit.params.index,
        "COEF": fit.params.values
    })

    return out, coef_tbl


# ----------------------------
# 1) Baseline run
# ----------------------------
baseline_out, coef_tbl = run_forecast_pipeline(y, X, horizon=h, scenario_name="BASELINE")

# Clean interpretability table (only exogenous coefficients)
coef_tbl_exog = coef_tbl[coef_tbl["PARAMETER"].isin(exog_cols)].copy()
coef_tbl_exog["ABS_COEF"] = coef_tbl_exog["COEF"].abs()
coef_tbl_exog = coef_tbl_exog.sort_values("ABS_COEF", ascending=False).drop(columns=["ABS_COEF"])

# ----------------------------
# 2) What-if scenario: Economic downturn (raise unemployment)
# ----------------------------
future_index = pd.date_range(start=y.index.max() + pd.Timedelta(weeks=1), periods=h, freq="W-FRI")
future_index.freq = "W-FRI"

last_row = X.iloc[-1]
econ_exog = pd.DataFrame([last_row.values] * h, columns=X.columns, index=future_index)

# Scenario: unemployment increases by +1.0 percentage point
econ_exog["UNEMPLOYMENT"] = econ_exog["UNEMPLOYMENT"] + np.linspace(0.2, 1.0, h)

econ_out, _ = run_forecast_pipeline(y, X, horizon=h, scenario_name="UNEMPLOYMENT_PLUS_1PT", scenario_exog_future=econ_exog)

# Combine
phase5_out = pd.concat([baseline_out, econ_out], ignore_index=True)

# Delta table
delta = econ_out[["DS", "Y_PRED"]].merge(
    baseline_out[["DS", "Y_PRED"]],
    on="DS",
    suffixes=("_ECON", "_BASE")
)
delta["DELTA_ECON_MINUS_BASE"] = delta["Y_PRED_ECON"] - delta["Y_PRED_BASE"]

print("Phase 5: automated pipeline + scenario simulation.")
print("\nExogenous coefficients (interpretability):")
display(coef_tbl_exog)

print("\nScenario impact preview (Unemployment +1pt minus Baseline):")
display(delta.head())

# ----------------------------
# Save outputs to Snowflake
# ----------------------------
session.create_dataframe(phase5_out).write.mode("overwrite").save_as_table("STORE_WEEKLY_FORECASTS_PHASE5")
session.create_dataframe(delta).write.mode("overwrite").save_as_table("STORE_WEEKLY_SCENARIO_DELTA_PHASE5")

print("\nSaved tables:")
print("- STORE_WEEKLY_FORECASTS_PHASE5")
print("- STORE_WEEKLY_SCENARIO_DELTA_PHASE5")


Phase 6 

The 12-week forecast for Store 1 should be treated as an operating plan for weekly labor, inventory replenishment, and promotional pacing. Operationally, the forecast provides a baseline expectation for weekly sales so the store can schedule staffing to match expected demand and avoid overstaffing during softer weeks or understaffing during stronger weeks. Inventory teams can use the forecast as a replenishment signal: when predicted weekly sales are higher, safety stock and order quantities should be increased slightly ahead of time to prevent stockouts, while lower predicted weeks can be used to reduce inbound volume and minimize carrying costs. From a retail execution standpoint, the forecast also supports calendar planning by identifying when sales are likely to be stronger and aligning promotion timing, end-cap space, and store readiness (labor coverage, truck unloading windows, and shelf availability) to those weeks.

The “what-if” scenario simulation adds a practical layer for planning under uncertainty. In the unemployment scenario (gradually increasing up to +1.0 point), the model indicates a proportional reduction in predicted sales relative to the baseline. This can be used as a contingency plan: if macroeconomic conditions deteriorate, leadership can proactively tighten purchasing, shift promotional strategy toward value-focused offers, and adjust labor budgets modestly to protect margin while maintaining customer experience. The key takeaway is not that unemployment causes an exact dollar change every week in real life, but that the model provides a structured way to stress-test the plan and quantify “how bad could it get” under a realistic external shift.

Decision support narrative

For stakeholders, the forecast and scenario outputs function as a weekly decision dashboard. Store leadership can use the baseline forecast as the primary weekly target for sales and staffing. The finance/planning view is to translate the forecast into budget guardrails (expected revenue band) and monitor weekly variance, especially in the final 12-week window where decisions are most actionable. For merchandising and operations, the forecast should drive a simple cadence: review next 4 weeks every week, confirm replenishment timing, and adjust labor schedules one to two weeks ahead based on the forecast direction and any known events. The scenario analysis supports executive conversations by providing a “Plan A vs Plan B” narrative: Plan A assumes conditions stay stable; Plan B assumes a measurable macro shift and recommends specific cost and inventory responses to reduce risk.

Risks and limitations

This forecast is only as strong as the data and assumptions behind it. First, the horizon is short (12 weeks) and the model relies on recent patterns; unexpected events, supply constraints, or sudden competitive changes can break historical relationships. Second, the scenario analysis assumes other drivers stay constant while unemployment changes; in reality, multiple variables move together and store-level customer behavior may respond nonlinearly. Third, some features may show weak effects (for example, the holiday flag was effectively near zero in the fitted model), which can happen when the signal is limited or already captured through seasonality; this means not every business intuition will appear as a strong coefficient. Finally, because the modeling was built for Store 1, the results should not automatically be generalized to other stores without repeating the same workflow and validating performance store-by-store.

Given these limitations, the recommendation is to treat the forecast as a decision aid rather than a guarantee, and to operationalize it through monitoring. Each week, compare actuals to forecast, track error metrics over time, and revisit the model if errors drift upward or if data conditions change. This keeps the forecasting workflow deployment-ready and aligned with business value